In [ ]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install pillow

In [1]:
# importing Libraries
import zipfile
import io
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from tensorflow.keras import layers

In [ ]:
# Generator model
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

In [ ]:
# dicriminator model
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
# Function to preprocess images
def preprocess_images(images, target_size=(64, 64)):
    processed_images = []

    # Ensure all images have the same shape
    max_height = max(image.shape[0] for image in images)
    max_width = max(image.shape[1] for image in images)

    for image in images:
        # Check if the image has the correct number of dimensions
        if len(image.shape) == 3:  # RGB image
            channels = image.shape[2]
            if channels != 3:
                raise ValueError("RGB images must have 3 channels.")
            processed_images.append(image)
        elif len(image.shape) == 2:  # Grayscale image
            processed_image = np.expand_dims(image, axis=-1)
            processed_images.append(processed_image)
        else:
            raise ValueError("Invalid image shape.")

    # Resize or pad images to have the same shape
    resized_images = []
    for i, image in enumerate(images):
        print(f"Processing image {i + 1}/{len(images)} with shape {image.shape}")
        if image.shape[0] != max_height or image.shape[1] != max_width:
            # Resize the image to the target size
            resized_image = tf.image.resize_with_pad(image, target_size[0], target_size[1])
            processed_images.append(resized_image.numpy())
        else:
            resized_images.append(image)

    return np.array(resized_images)

In [ ]:
# Open the ZIP file in read mode
with zipfile.ZipFile('/content/drive/MyDrive/art_dataset.zip', 'r') as zip_file:
    image_files = zip_file.namelist()
    images = []

    # Read image data from each file in the ZIP file
    for file_name in image_files:
        with zip_file.open(file_name) as image_file:
            # Read image data from the file-like object
            image_data = io.BytesIO(image_file.read())
            # Open the image using PIL (Python Imaging Library)
            image = Image.open(image_data)
            # Convert the image to a numpy array
            image_array = np.array(image)
            images.append(image_array)

print(images[0].shape)
print("Number of images:", len(images))
print("Shape of first image:", images[2000].shape)

# Preprocess images
preprocessed_images = preprocess_images(images)

# Instantiate the generator and discriminator models
generator = make_generator_model()
discriminator = make_discriminator_model()

# Print the summary of the models
generator.summary()
discriminator.summary()


In [ ]:
# Define the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model

# Training function
def train_gan(gan, generator, discriminator, dataset, latent_dim, epochs, batch_size):
    for epoch in range(epochs):
        for batch in dataset:
            # Generate fake images
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_images = generator.predict(noise)

            # Combine real and fake images
            real_images = batch
            images = np.concatenate([real_images, fake_images])

            # Labels for real and fake images
            real_labels = np.ones((batch_size, 1))
            fake_labels = np.zeros((batch_size, 1))
            labels = np.concatenate([real_labels, fake_labels])

            # Train discriminator
            d_loss = discriminator.train_on_batch(images, labels)

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, real_labels)

        print(f"Epoch {epoch + 1}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")

In [ ]:
# Save generated images function
def save_generated_images(generator, epoch, latent_dim, save_path, num_samples=25):
    noise = np.random.normal(0, 1, (num_samples, latent_dim))
    generated_images = generator.predict(noise)

    # Rescale images to range [0, 255]
    generated_images = 127.5 * generated_images + 127.5

    # Save generated images as a grid
    rows = int(np.sqrt(num_samples))
    cols = num_samples // rows
    grid = np.zeros((rows * generated_images.shape[1], cols * generated_images.shape[2], 3), dtype=np.uint8)
    for i in range(rows):
        for j in range(cols):
            image = generated_images[i * cols + j]
            grid[i * generated_images.shape[1]:(i + 1) * generated_images.shape[1],
                 j * generated_images.shape[2]:(j + 1) * generated_images.shape[2]] = image.astype(np.uint8)

    # Save grid of images
    filename = f"generated_images_epoch_{epoch}.png"
    tf.keras.preprocessing.image.save_img(save_path + filename, grid)

In [ ]:
#main function
def main():
    # Set random seed for reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)

    # Define parameters
    latent_dim = 100
    image_shape = (28, 28, 1)
    epochs = 8000
    batch_size = 128
    save_interval = 10
    save_path = "generated_images/"

    # Build and compile models
    generator = make_generator_modelnerator(latent_dim, image_shape)
    discriminator = make_discriminator_modelr(image_shape)
    gan = build_gan(generator, discriminator)


    # Train GAN
    train_gan(gan, generator, discriminator, dataset, latent_dim, epochs, batch_size)

    # Save generated images at specified intervals during training
    for epoch in range(0, epochs + 1, save_interval):
        save_generated_images(generator, epoch, latent_dim, save_path)

if __name__ == "__main__":
    main()